In [66]:
import pandas as pd
import sqlite3, hashlib, requests, json, uuid

# Collect to and query the DB for all of the peers
conn = sqlite3.connect('../test.db')
query = "SELECT * FROM peers;"

df = pd.read_sql_query(query,conn)

In [50]:
# Total Unique IPs
print(f"Total Unique IPs: {len(df.address.unique())} Total IPs: {len(df.address)}")

# Getting a df of unique IP+torrent combos only
## Hash the IP + torrent name concatenated
## Insert hash as own column
## Select only unique hashes

df["ip_tor_hash"] = df["address"] + df["torrent_name"]
df["ip_tor_hash"] = df["ip_tor_hash"].apply(lambda x: hashlib.md5(x.encode()).hexdigest())

u_df = df.drop_duplicates(subset=['ip_tor_hash']) # Unique df

Total Unique IPs: 114 Total IPs: 1044


In [63]:
# Get IP Address information from ipinfo.io
## e00976a117244e
## Get the responses into a larger object that we can iterate through after the queries

ips = {}

for ip in u_df.address:
    r = requests.get(f"https://ipinfo.io/{ip}?token=e00976a117244e")
    data = json.loads(r.text)
    ips[ip] = data
    #break

In [68]:
# Save our IPs to a file for now so we do not lose them and waste API calls on duplications
with open(f"ips_{str(uuid.uuid4())}.json", "w") as f:
    f.write(json.dumps(ips))
print(ips)


In [118]:
# Write a parser for the IP entries that returns a dictionary for the
# Calling function to then insert into the df
def ip_parser(ip):
    """
    Takes one IP from the ips dictionary that was created.
    ip.addr.is.here : {
        ip: ip.addr.is.here,
        county: etc....
    }
    """
    cleaned_ip = {
        "ip": "", "hostname": "", "city": "", 
        "region": "", "country": "", "loc": "",
        "org": "", "postal": "", "timezone": ""
        }
    
    for k,v in ip.items():
        cleaned_ip[k] = v
        
    return { cleaned_ip["ip"]: {
        "address": cleaned_ip["ip"],
        "hostname": cleaned_ip["hostname"],
        "city": cleaned_ip["city"],
        "region": cleaned_ip["region"],
        "country": cleaned_ip["country"],
        "loc": cleaned_ip["loc"],
        "org": cleaned_ip["org"],
        "postal": cleaned_ip["postal"],
        "timezone": cleaned_ip["timezone"]
        }
    }


In [119]:
# Enter default empty strings for missing values for data clarity. Convert into its own dataframe for insertion into main DF
clean_ip_list = {}
for ip in ips:    
    clean_ip_list[ip] = ip_parser(ips[ip])[ip]
    
ipdf = pd.DataFrame.from_dict(clean_ip_list).T

In [123]:
# Merge into a single df with u_df
mu_df = pd.merge(u_df, ipdf, on="address")

In [130]:
mu_df.sample(8)

,address,clientIsChoked,clientIsInterested,clientName,flagStr,isDownloadingFrom,isEncrypted,isIncoming,isUTP,isUploadingTo,...,torrent_name,ip_tor_hash,hostname,city,region,country,loc,org,postal,timezone
11,91.219.215.22,True,False,qBittorrent 4.3.9,I,False,False,True,False,False,...,Rocky-8.5-x86_64-dvd1,edcde7d7d51c3f43bf87c0c92be27c49,,Oslo,Oslo,NO,"59.9127,10.7461",AS9009 M247 Ltd,0001,Europe/Oslo
91,78.71.213.167,True,False,Transmission 3.00,X,False,False,False,False,False,...,Rocky-8.5-x86_64-dvd1,ab972c94ac86cf921320fb5004fdc5d5,78-71-213-167-no2000.tbcn.telia.com,Mölndal,Västra Götaland,SE,"57.6554,12.0138",AS3301 Telia Company AB,431 00,Europe/Stockholm
16,189.0.199.128,True,False,Transmission 2.93,?EHI,False,True,True,False,False,...,2021-10-30-raspios-bullseye-arm64-lite.zip,93be98a55bf8be0c1c8dcf0a38f9b62c,ip-189-0-199-128.user.vivozap.com.br,São Paulo,São Paulo,BR,"-23.5475,-46.6361",AS27699 TELEFÔNICA BRASIL S.A,01000-000,America/Sao_Paulo
77,185.65.134.165,True,False,Transmission 3.00,EI,False,True,True,False,False,...,Rocky-8.5-x86_64-dvd1,7b88f011b56d479edca52d83c81caff3,,Amsterdam,North Holland,NL,"52.3740,4.8897",AS39351 31173 Services AB,1012,Europe/Amsterdam
33,138.34.73.91,True,False,µTorrent 3.5.5,uI,False,False,True,False,False,...,Rocky-8.5-x86_64-dvd1,a6cded076cb6d004c49e063553e247aa,,Cache Bay,Ontario,CA,"46.3668,-79.9830",AS5690 Vianet,P2B,America/Toronto
24,82.83.153.148,True,False,Transmission 3.00,X,False,False,False,False,False,...,Rocky-8.5-x86_64-dvd1,54f4fefcaa9ed066d6bd819b4382e52d,dslc-082-083-153-148.pools.arcor-ip.net,Sinsheim,Baden-Württemberg,DE,"49.2529,8.8787",AS3209 Vodafone GmbH,74889,Europe/Berlin
26,62.178.64.251,True,False,Transmission 3.00,E,False,True,False,False,False,...,Rocky-8.5-x86_64-dvd1,a4669cd9de65d485fde265d4a410218d,62-178-64-251.cable.dynamic.surfer.at,Vienna,Vienna,AT,"48.2085,16.3721",AS8412 T-Mobile Austria GmbH,1010,Europe/Vienna
74,77.221.60.172,True,False,Transmission 2.94,EI,False,True,True,False,False,...,Rocky-8.5-x86_64-dvd1,baae2fd4f4a37b68cda65a240b4875ec,,Kondoros,Bekes County,HU,"46.7667,20.8000",AS30836 23VNet Kft.,5553,Europe/Budapest


,address,clientIsChoked,clientIsInterested,clientName,flagStr,isDownloadingFrom,isEncrypted,isIncoming,isUTP,isUploadingTo,...,torrent_name,ip_tor_hash,hostname,city,region,country,loc,org,postal,timezone
0,14.187.40.246,True,False,Transmission 3.00,?EI,False,True,True,False,False,...,Rocky-8.5-x86_64-dvd1,eca05aaeb4a9f9ecaefd9e9df3fce818,static.vnpt.vn,Ho Chi Minh City,Ho Chi Minh,VN,"10.8230,106.6296",AS45899 VNPT Corp,71606,Asia/Ho_Chi_Minh
1,47.197.200.241,False,False,qBittorrent 4.2.1,uKI,False,False,True,False,False,...,Rocky-8.5-x86_64-dvd1,cc40bf88a668a8cb211c067a48332054,,Tampa,Florida,US,"27.9475,-82.4584","AS5650 Frontier Communications of America, Inc.",33606,America/New_York
2,81.221.215.194,True,False,qBittorrent 4.3.9,UXI,False,False,True,False,True,...,Rocky-8.5-x86_64-dvd1,d27128189ad18f4f1729419da6d5e46e,194-215-221-81.pool.dsl-net.ch,Zürich,Zurich,CH,"47.3667,8.5500",AS1836 green.ch AG,8000,Europe/Zurich
3,98.29.27.26,False,False,qBittorrent 4.3.9,KI,False,False,True,False,False,...,Rocky-8.5-x86_64-dvd1,f0b44d9ccd371c9fe12121e2ddb0e063,mta-98-29-27-26.neo.rr.com,Dayton,Ohio,US,"39.6464,-84.1717",AS10796 Charter Communications Inc,45459,America/New_York
4,107.175.102.53,True,False,Transmission 3.00,?EI,False,True,True,False,False,...,Rocky-8.5-x86_64-dvd1,2970c2522e8b819671960d02d69eeae5,53-102-175-107.reverse-dns,Buffalo,New York,US,"42.8865,-78.8784","AS20278 Nexeon Technologies, Inc.",14202,America/New_York
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,5.79.68.181,True,False,libTorrent (Rakshasa) 0.13.6,E,False,True,False,False,False,...,Rocky-8.5-x86_64-dvd1,94987155a0ff7966dec32a357eb89ddc,,Amsterdam,North Holland,NL,"52.3740,4.8897",AS60781 LeaseWeb Netherlands B.V.,1012,Europe/Amsterdam
111,51.174.215.38,True,False,Transmission 3.00,E,False,True,False,False,False,...,2021-10-30-raspios-bullseye-arm64-lite.zip,deb1a57d0fd4667d3ca8a25008bca75b,38.51-174-215.customer.lyse.net,Skien,Vestfold og Telemark,NO,"59.2096,9.6090",AS29695 Altibox AS,3715,Europe/Oslo
112,81.200.17.253,True,False,Transmission 3.00,EI,False,True,True,False,False,...,2021-10-30-raspios-bullseye-arm64-lite.zip,6db927d54b829e4fb25a16a692dd7f4d,,Zheleznodorozhnyy,Moscow Oblast,RU,"55.7440,38.0168",AS29124 Iskratelecom CJSC,143980,Europe/Moscow
113,62.217.191.106,True,False,Transmission 2.92,EI,False,True,True,False,False,...,Rocky-8.5-x86_64-dvd1,d1caa4b06d0b7d260c60a071b017d361,,Moscow,Moscow,RU,"55.7522,37.6156",AS41275 Lovitel LLC,101000,Europe/Moscow
